In [48]:
import psycopg2
import pandas as pd
import os
import uuid
from dotenv import load_dotenv
from sqlalchemy import create_engine
import numpy as np

# Load environment variables from a .env file
load_dotenv()

# Define your CSV file path
csv_file_path = '../data/countries_epis.csv'  # Update to the correct path

# Establish a database connection
try:
    conn = psycopg2.connect(
        dbname=os.getenv("DB_NAME"),
        user=os.getenv("DB_USER"),
        password=os.getenv("DB_PASSWORD"),
        host=os.getenv("DB_HOST"),
        port=os.getenv("DB_PORT")
    )
    # Create a cursor
    cursor = conn.cursor()

    # Create a database engine
    db_uri = f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"
    engine = create_engine(db_uri)

    # Read the entire countries table into a DataFrame
    countries_df = pd.read_sql_query("SELECT * FROM countries", engine)

    # Read the epis table into a DataFrame
    epis_df = pd.read_sql_query("SELECT * FROM epis", engine)

    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_file_path)

    # Prepare the SQL statement using an f-string with multiline string
    sql_statement = f"""
       INSERT INTO countries_epis (country_epi_id, country_id, epi_id, score, rank, change, year, created, updated)
       VALUES (%s, %s, %s, %s, %s, %s, %s, CURRENT_TIMESTAMP, CURRENT_TIMESTAMP)
    """

    if not df.empty and not countries_df.empty and not epis_df.empty:
      for index, row in df.iterrows():
         # Get the country_id from the countries DataFrame
         code = row['code']
         country_id = countries_df.loc[countries_df['country_code'] == code, 'country_id'].values[0]
         print(countries_df.loc[countries_df['country_code'] == code, 'name'].values[0])
         # Iterate over the desired EPIs
         epi_columns = ['BHV', 'SHI', 'SPI', 'PAR', 'MPA', 'RMS']

         for epi_column in epi_columns:
               print(epi_column)
               if epi_column in df.columns:
                  epi_score = row.get(epi_column, 0.0)
                  epi_rank = row.get(f'{epi_column}.rnk', -1)
                  epi_change = row.get(f'{epi_column}.change', 0.0)

                  if np.isnan(epi_rank):
                     epi_rank = -1

                  epi_year = 2022
                  # Get the epi_id using the extracted column name
                  epi_id = epis_df.loc[epis_df['abbreviation'] == epi_column, 'epi_id'].values[0]

                  # Generate a UUID for country_epi_id
                  country_epi_id = uuid.uuid4()

                  print(country_epi_id, country_id, epi_id, epi_score, epi_rank, epi_change, epi_year)

                  # Insert the data into countries_epis using the prepared statement
                  cursor.execute(
                     sql_statement,
                     (country_epi_id, country_id, epi_id, epi_score, epi_rank, epi_change, epi_year)
                  )

    # Commit the changes to the database
    conn.commit()

except Exception as e:
    print(f"Error: {e}")

finally:
    # Close the connection
    if conn:
        conn.close()



Afghanistan
BHV
ed37df8a-9d86-4496-afac-4b721b16b203 606e8692-b533-4311-b98a-abcce7a4938b 6d88f145-a1be-45ac-96d7-122ec8342815 41.9 101 0 2022
SHI
9f74507d-3f55-4d08-9e45-fb4f06601657 606e8692-b533-4311-b98a-abcce7a4938b 0d1ea395-50ce-4813-a4a9-70f55faa7720 90.8 54.0 -4.7 2022
SPI
a29b402f-477a-47ac-b3d1-b99f045eed55 606e8692-b533-4311-b98a-abcce7a4938b e8c099a8-3313-45a1-b1be-6a97a5ed609f 13.4 155.0 12.5 2022
PAR
6ff8fea2-7d53-46a4-b81c-b22802c6fa1c 606e8692-b533-4311-b98a-abcce7a4938b e849fbd1-a4e3-4923-a973-1fc25cbf3d1e 5.2 166 2.1 2022
MPA
352cb22f-5976-4227-bc04-d3055625266c 606e8692-b533-4311-b98a-abcce7a4938b cee4d2c4-f639-4539-8df7-d5a3022babc2 nan -1 nan 2022
RMS
369ac69b-e068-461f-afd1-f97649de6169 606e8692-b533-4311-b98a-abcce7a4938b 5b038d3a-596b-4ea2-8ee6-ef96e40a7f84 nan -1 nan 2022
Angola
BHV
5b355a02-67b2-4ecf-af97-5be5ff242ac7 6ab89298-f2be-46c9-92e0-9e2457553d1c 6d88f145-a1be-45ac-96d7-122ec8342815 61.5 22 0 2022
SHI
89c89c30-86d5-4e7c-8b8c-33ab2bc557cb 6ab89298-f2be-